# Vanilla closed-form with xtensor-python

This notebook demonstrates the usage of a python extension module built upon **xtensor-python**. The following packages are required to get it work:
- xtl
- xtensor
- xtensor-python
- pybind11
- numpy
- bqplot
- ipywidgets

We suggest to insall them with conda: `conda install xtensor-python bqplot ipyvolume -c quantstack -c conda-forge`
so the dependencies are handled for you.

In [63]:
import math
import numpy as np
import xtensor_closed_forms as xcf
from bqplot import (LinearScale, Lines, Axis, Figure)
import ipywidgets as widgets
import ipyvolume as ipv
from IPython.display import display

## 1. Simple european call

In this section, we plot the price of a european call depending on the spot, and show how the volatility, maturity and rates
can influence this price curve. We can pass numpy arrays to the functions defined in `xtensor_closed_forms` even if these are not python functions. Besides, `xtensor-python` operates on numpy arrays **in place**, so the arrays are never copied.

In [10]:
iscall = True
vol = 0.2
mat = 1.
rate = 0.04
strike = 1.
spot = np.arange(0.1, 1.9, 0.01)

In [3]:
# Documentation works as if it was a python function
?xcf.vanilla_discounted_payoff

In [4]:
price = xcf.bs_discounted_price(spot, strike, vol, mat, rate, iscall)
discounted_payoff = xcf.vanilla_discounted_payoff(spot, strike, mat, rate, iscall)

In [9]:
sc_x = LinearScale()
sc_y = LinearScale(max=1.)
call_graph = Lines(x=spot, y=price, scales={'x': sc_x, 'y': sc_y}, labels=['Price'], display_legend=True)
payoff_graph = Lines(x=spot, y=discounted_payoff, scales={'x': sc_x, 'y': sc_y}, labels=['Payoff'], colors=['red'],
                    display_legend=True)
ax_x = Axis(scale=sc_x, label="spot")
ax_y = Axis(scale=sc_y, orientation='vertical', label="price")

vol_slider = widgets.FloatSlider(value=vol, min=0, max=1, step=0.05, description='volatility')
def handle_vol_change(change):
    global vol
    vol = change.new
    call_graph.y = xcf.bs_discounted_price(spot, strike, vol, mat, rate, iscall)
vol_slider.observe(handle_vol_change, names='value')

rate_slider = widgets.FloatSlider(value=rate, min=0, max=0.1, step=0.01, description='rate')
def handle_rate_change(change):
    global rate
    rate = change.new
    call_graph.y = xcf.bs_discounted_price(spot, strike, vol, mat, rate, iscall)
    payoff_graph.y = xcf.vanilla_discounted_payoff(spot, strike, mat, rate, iscall)
rate_slider.observe(handle_rate_change, names='value')

mat_slider = widgets.FloatSlider(value=mat, min=0.5, max=10., step=0.5, description='maturity')
def handle_mat_change(change):
    global mat
    mat = change.new
    call_graph.y = xcf.bs_discounted_price(spot, strike, vol, mat, rate, iscall)
    payoff_graph.y = xcf.vanilla_discounted_payoff(spot, strike, mat, rate, iscall)
mat_slider.observe(handle_mat_change, names='value')

figure = Figure(marks=[call_graph, payoff_graph], axes=[ax_x, ax_y], title='European Call',
               legend_location='top-left')

r = widgets.VBox([figure, vol_slider, rate_slider, mat_slider])
display(r)

## 2. Surface plot for many calls

In this section, we demonstrate the braodcasting feature available in `xtensor-python`. To do so, we compute the price matrix of european calls for different spots and volatilities.

In [77]:
#This avoids having the 3D plot updated when changing the sliders of the 2D plot (and vice-versa)
iscall_2d = True
mat_2d = 1.
rate_2d = 0.04
strike_2d = 1.
spot_2d = np.arange(0.1, 1.5, 0.01)

In [79]:
vol_lin = np.arange(0.05, 0.35, 0.01)
vol_2d = vol_lin[:, np.newaxis]
price_2d = xcf.bs_discounted_price(spot_2d, strike_2d, vol_2d, mat_2d, rate_2d, iscall_2d)

In [76]:
fig = ipv.figure()
ipv.pylab.ylim(0, 0.6)
ipv.pylab.zlim(0, 0.4)
ipv.pylab.xlabel('spot')
ipv.pylab.ylabel('price')
ipv.pylab.zlabel('volatility')
x, y = np.meshgrid(spot_2d, vol_2d)
ipv.plot_mesh(x, price_2d, y)

rate_slider_2d = widgets.FloatSlider(value=rate, min=0, max=0.1, step=0.01, description='rate')
def handle_rate_change_2d(change):
    global rate_2d
    global price_2d
    rate_2d = change.new
    price_2d = xcf.bs_discounted_price(spot_2d, strike_2d, vol_2d, mat_2d, rate_2d, iscall_2d)
    mesh = ipv.plot_mesh(x, price_2d, y)
    fig.meshes = [mesh]
rate_slider_2d.observe(handle_rate_change_2d, names='value')

mat_slider_2d = mat_slider = widgets.FloatSlider(value=mat, min=0.5, max=10., step=0.5, description='maturity')
def handle_mat_change_2d(change):
    global mat_2d
    global price_2d
    mat_2d = change.new
    price_2d = xcf.bs_discounted_price(spot_2d, strike_2d, vol_2d, mat_2d, rate_2d, iscall_2d)
    mesh = ipv.plot_mesh(x, price_2d, y)
    fig.meshes = [mesh]
mat_slider_2d.observe(handle_mat_change_2d, names='value')

r_2d = widgets.VBox([fig, rate_slider_2d, mat_slider_2d])
r_2d